In [4]:
# Disaster Mapping and Visualization Tool for UN DSS - Myanmar Earthquake (Advanced Version)

# --- Smart Auto Installer for Missing Libraries ---
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for pkg in ['folium', 'pandas', 'geopandas', 'ipywidgets']: 
    install_if_missing(pkg)

# --- Import Libraries ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display, clear_output
import geopandas as gpd

# --- Initial Mock Data (only if CSV not uploaded) ---
def load_default_data():
    return pd.DataFrame({
        'Location': ['Mandalay', 'Pyin Oo Lwin', 'Sagaing', 'Naypyidaw', 'Meiktila'],
        'Latitude': [21.9781, 22.0355, 21.8787, 19.7633, 20.8778],
        'Longitude': [96.0836, 96.4597, 95.9784, 96.0785, 95.8585],
        'Damage_Level': ['Severe', 'Moderate', 'Severe', 'Mild', 'Moderate'],
        'Needs': ['Medical, Shelter', 'Food, Water', 'Medical, Water', 'Temporary Shelter', 'Food, Medical'],
        'IDPs_Affected': [100000, 50000, 120000, 25000, 40000]
    })

# Load initial data
df = load_default_data()

# --- UI Components ---

upload_widget = widgets.FileUpload(
    accept='.csv', 
    multiple=False,
    description='Upload CSV',
    style={'description_width': 'initial'}
)

damage_filter = widgets.Dropdown(
    options=['All', 'Mild', 'Moderate', 'Severe'],
    value='All',
    description='Damage:',
    style={'description_width': 'initial'}
)

location_search = widgets.Text(
    value='',
    placeholder='Enter city name...',
    description='Search Location:',
    style={'description_width': 'initial'}
)

refresh_button = widgets.Button(
    description='Update Map',
    button_style='success',
    tooltip='Click to refresh map based on filters'
)

map_output = widgets.Output()

# --- Map Plot Function ---
def plot_map(damage_filter_value, location_query):
    m = folium.Map(location=[21.5, 96.0], zoom_start=6)
    marker_cluster = MarkerCluster().add_to(m)

    filtered_df = df.copy()

    if damage_filter_value != 'All':
        filtered_df = filtered_df[filtered_df['Damage_Level'] == damage_filter_value]

    if location_query:
        filtered_df = filtered_df[filtered_df['Location'].str.contains(location_query, case=False)]

    color_dict = {'Severe': 'red', 'Moderate': 'orange', 'Mild': 'green'}

    for idx, row in filtered_df.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=(f"<b>Location:</b> {row['Location']}<br>"
                   f"<b>Damage:</b> {row['Damage_Level']}<br>"
                   f"<b>Needs:</b> {row['Needs']}<br>"
                   f"<b>IDPs Affected:</b> {row['IDPs_Affected']:,}"),
            icon=folium.Icon(color=color_dict.get(row['Damage_Level'], 'blue'))
        ).add_to(marker_cluster)

    # 👉 ADD HeatMap Layer here 👈
    from folium.plugins import HeatMap

    heat_data = [
        [row['Latitude'], row['Longitude'], row['IDPs_Affected']] 
        for idx, row in filtered_df.iterrows()
    ]

    HeatMap(heat_data, radius=20, blur=15, max_zoom=6).add_to(m)

    return m


# --- Button Event ---
def on_refresh_clicked(b):
    with map_output:
        clear_output(wait=True)
        global df
        
        if upload_widget.value:
            content = list(upload_widget.value.values())[0]['content']
            df = pd.read_csv(pd.io.common.BytesIO(content))

        m = plot_map(damage_filter.value, location_search.value)
        display(m)
        m.save('myanmar_earthquake_map.html')

refresh_button.on_click(on_refresh_clicked)

# --- Display UI ---
controls = widgets.VBox([upload_widget, damage_filter, location_search, refresh_button])
display(widgets.HBox([controls]))
display(map_output)

# --- Initialize Map Once ---
on_refresh_clicked(None)


Output()